In [ ]:
import netCDF4
import numpy as np

# Loading the data from the net CDF files.
There were 60 files and the code below loops through them and sorts the data into the correct variables.

In [ ]:
for j in range(1,6):
    for i in range(0,24,2):
        nc=netCDF4.Dataset('python_generated_ml_cape_training_data_2020010%s-2020010%s_%s.nc'%(j,j,i))
        if i==0:
          Temp_values=nc.variables['field'][:][0:70,:]
          q_values=nc.variables['field'][:][70:140,:]
          p_values=nc.variables['field'][:][140:210,:]
          LWC=nc.variables['field'][:][263:333,:]
          IWC=nc.variables['field'][:] [333:403,:]
          ancils=nc.variables['field'][:][210:214,:]
          CFP=nc.variables['field'][:][403:473,:]
        else:
            Temp_values=np.append(Temp_values,nc.variables['field'][:][0:70,:],axis=1)
            q_values=np.append(q_values,nc.variables['field'][:][70:140,:],axis=1)
            p_values=np.append(p_values,nc.variables['field'][:][140:210,:],axis=1)
            LWC=np.append(LWC,nc.variables['field'][:][263:333,:],axis=1)
            IWC=np.append(IWC,nc.variables['field'][:][333:403,:],axis=1)
            ancils=np.append(ancils,nc.variables['field'][:][210:214,:],axis=1)
            CFP=np.append(CFP,nc.variables['field'][:][403:473,:],axis=1)
    if j==1:
        final_Temp_values=Temp_values
        final_q_values=q_values
        final_p_values=p_values
        final_IWC=IWC
        final_LWC=LWC
        final_ancils=ancils
        final_CFP=CFP
    else:
        final_Temp_values=np.append(final_Temp_values,Temp_values,axis=1)
        final_q_values=np.append(final_q_values,q_values,axis=1)
        final_p_values=np.append(final_p_values,p_values,axis=1)
        final_LWC=np.append(final_LWC,LWC,axis=1)
        final_IWC=np.append(final_IWC,IWC,axis=1)
        final_ancils=np.append(final_ancils,ancils,axis=1)
        final_CFP=np.append(final_CFP,CFP,axis=1)      

# Undoing the normalisation on the data
The data came with a rough normalisation, this was undone in order to do calculations with the corrct values. 

In [ ]:
Unnormalised_Temp_values=final_Temp_values*180+140
Unnormalised_q_values=final_q_values*0.025
Unnormalised_p_values=final_p_values*106000
Unnormalised_LWC=final_LWC*5e-3
Unnormalised_IWC=final_IWC*5e-3
final_ancils[0,:]*=4000
final_ancils[1,:]*=1200
Unnormalised_ancils=final_ancils

# Correcting the humidity and temperature

In [ ]:
newq=Unnormalised_q_values+Unnormalised_LWC+Unnormalised_IWC
lv       = 2.501e6
lf       = 2.834e6
cp       = 1005.0
lvovercp = lv / cp
lfovercp = lf / cp
newTemp=Unnormalised_Temp_values-(lvovercp*Unnormalised_LWC) - (lfovercp*Unnormalised_IWC)

# Collecting the input data into a single array

In [ ]:
Inputs=newTemp
Inputs=np.append(Inputs,newq,axis=0)
Inputs=np.append(Inputs,Unnormalised_p_values,axis=0)
Inputs=np.append(Inputs,Unnormalised_ancils,axis=0)

# Transposing the inputs and outputs so they have the right shape

In [ ]:
Inputs=Inputs.transpose()

In [ ]:
Outputs=final_CFP.transpose()

# Shuffling the data
The permutation used for the project is given in a npz file in the repository. The code below will load the permutation and perform the shuffle if permutation.npz is in the same folder as this notebook.

In [ ]:
load=np.load('permutation.npz')

In [ ]:
perm=load['arr_0']

In [ ]:
for i in range(0,214):
  Inputs[:,i]=Inputs[:,i][perm]
for i in range(0,70):
  Outputs[:,i]=Outputs[:,i][perm]

# Saving the inputs and outputs
This saves them so they can be used in other notebooks

In [ ]:
np.savez('shuffled_inputs', Inputs)

In [ ]:
np.savez('shuffled_outputs', Outputs)